# Apply this code to sample TS from model.

In [1]:
# --- Importing and defining some functions ----
import os
import torch
import py3Dmol
import numpy as np

from typing import Optional
from torch import tensor
from e3nn import o3
from torch_scatter import scatter_mean

from oa_reactdiff.model import LEFTNet

default_float = torch.float64
torch.set_default_dtype(default_float)  # Use double precision for more accurate testing


def remove_mean_batch(
    x: tensor, 
    indices: Optional[tensor] = None
) -> tensor:
    """Remove the mean from each batch in x

    Args:
        x (tensor): input tensor.
        indices (Optional[tensor], optional): batch indices. Defaults to None.

    Returns:
        tensor: output tensor with batch mean as 0.
    """
    if indices == None:
         return x - torch.mean(x, dim=0)
    mean = scatter_mean(x, indices, dim=0)
    x = x - mean[indices]
    return x


def draw_in_3dmol(mol: str, fmt: str = "xyz") -> py3Dmol.view:
    """Draw the molecule

    Args:
        mol (str): str content of molecule.
        fmt (str, optional): format. Defaults to "xyz".

    Returns:
        py3Dmol.view: output viewer
    """
    viewer = py3Dmol.view(1024, 576)
    viewer.addModel(mol, fmt)
    viewer.setStyle({'stick': {}, "sphere": {"radius": 0.36}})
    viewer.zoomTo()
    return viewer


def assemble_xyz(z: list, pos: tensor) -> str:
    """Assembling atomic numbers and positions into xyz format

    Args:
        z (list): chemical elements
        pos (tensor): 3D coordinates

    Returns:
        str: xyz string
    """
    natoms =len(z)
    xyz = f"{natoms}\n\n"
    for _z, _pos in zip(z, pos.numpy()):
        xyz += f"{_z}\t" + "\t".join([str(x) for x in _pos]) + "\n"
    return xyz

### Building a LEFTNet model

A simple test is performed to verify SE(3) symmetry. The model here is for testing, so we only need to build a very small model.

Note: [LEFTNet](https://arxiv.org/abs/2304.04757) is a new SOTA-level SE(3) graph neural network. Although we use LEFTNet here, the properties it exhibits are model-independent (other SE(3) models, such as [EGNN](https://arxiv.org/pdf/2102.09844.pdf), will give the same results)

TL: EGNN is not SE$(3)$  equivariant?

In [2]:
num_layers = 2
hidden_channels = 8
in_hidden_channels = 4
num_radial = 4

model =  LEFTNet(
    num_layers=num_layers,
    hidden_channels=hidden_channels,
    in_hidden_channels=in_hidden_channels,
    num_radial=num_radial,
    object_aware=False,
)

sum(p.numel() for p in model.parameters() if p.requires_grad)

/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "


7882



### Create an "Object-Aware" LEFTNet

In [3]:
# --- Importing necessary function ---
from torch.utils.data import DataLoader

from oa_reactdiff.trainer.pl_trainer import DDPMModule


from oa_reactdiff.dataset import ProcessedTS1x, ProcessedSCAN
from oa_reactdiff.diffusion._schedule import DiffSchedule, PredefinedNoiseSchedule

from oa_reactdiff.diffusion._normalizer import FEATURE_MAPPING
from oa_reactdiff.analyze.rmsd import batch_rmsd

from oa_reactdiff.utils.sampling_tools import assemble_sample_inputs, write_tmp_xyz

In [4]:
!pwd

/misc/home/guest50/OAReactDiff



### Import the pre-trained model and redefine the schedule.

In [6]:
# TL fix: {
from oa_reactdiff.trainer.pl_trainer import ConfModule, DDPMModule
# } fix. Why didn' this carry over from the previous cell import statement?

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda")
print(device) # TL

tspath = os.path.abspath(os.path.join(os.getcwd(), "oa_reactdiff","trainer"))
print(tspath)
# zenodo_pretrained_ckpt
ddpm_trainer = ConfModule.load_from_checkpoint(
    checkpoint_path="./trained_models/TSDiffusion-TS1x-All-Confidence-epoch1999.ckpt",
    #checkpoint_path=os.path.abspath("/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer/checkpoint/OAReactDiff/leftnet-0-84724de349da/ddpm-epoch=1964-val-totloss=510.42.ckpt"),
    #checkpoint_path=os.path.abspath("/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer/checkpoint/OAReactDiff/leftnet-0-84724de349da/ddpm-epoch=1999-val-totloss=523.77.ckpt"),
    #checkpoint_path="./pretrained-ts1x-diff.ckpt", # original
    #checkpoint_path=f"{tspath}/checkpoint/OAReactDiff/leftnet-0-f1ff7dc18fa3/ddpm-epoch=1999-val-totloss=509.31.ckpt", # Our recapitulation
    map_location=device,
)
ddpm_trainer = ddpm_trainer.to(device)

cuda
/misc/home/guest50/OAReactDiff/oa_reactdiff/trainer


/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torch_geometric/nn/conv/message_passing.py:972: UserWarning: 'EquiMessage.jittable' is deprecated and a no-op. Please remove its usage.
  warnings.warn(f"'{self.__class__.__name__}.jittable' is deprecated "
/misc/home/guest50/micromamba/envs/oa_reactdiff_m/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


# Confidence model cannot do inpainting, why was the official version better?

In [9]:
# Tangent: lookk at checkpoint metadata:
conf_ckpt = torch.load("./trained_models/TSDiffusion-TS1x-All-Confidence-epoch1999.ckpt", map_location="cpu")
conf_ckpt.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [10]:
conf_ckpt["epoch"]

1999

In [11]:
zenodo_TS1x_pretrain_ckpt = torch.load("./trained_models/pretrained-ts1x-diff-zenodo.ckpt", map_location="cpu")

In [12]:
zenodo_TS1x_pretrain_ckpt.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [13]:
zenodo_TS1x_pretrain_ckpt["epoch"]

2075

In [14]:
zenodo_TS1x_pretrain_ckpt["global_step"]

147325

In [15]:
zenodo_TS1x_pretrain_ckpt["pytorch-lightning_version"]

'1.8.6'

In [23]:
list(zenodo_TS1x_pretrain_ckpt["state_dict"].keys())[:10]

['ddpm.dynamics.model.embedding.weight',
 'ddpm.dynamics.model.embedding.bias',
 'ddpm.dynamics.model.embedding_out.weight',
 'ddpm.dynamics.model.embedding_out.bias',
 'ddpm.dynamics.model.radial_emb.means',
 'ddpm.dynamics.model.radial_emb.betas',
 'ddpm.dynamics.model.neighbor_emb.embedding.weight',
 'ddpm.dynamics.model.neighbor_emb.embedding.bias',
 'ddpm.dynamics.model.s2v.lin1.0.weight',
 'ddpm.dynamics.model.s2v.lin1.0.bias']

In [27]:
zenodo_TS1x_pretrain_ckpt["loops"].keys()

dict_keys(['fit_loop', 'validate_loop', 'test_loop', 'predict_loop'])

In [30]:
list(zenodo_TS1x_pretrain_ckpt["callbacks"].keys())

["EarlyStopping{'monitor': 'val-totloss', 'mode': 'min'}",
 'EMACallback',
 "ModelCheckpoint{'monitor': 'val-totloss', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None, 'save_on_train_epoch_end': True}"]

In [33]:
zenodo_TS1x_pretrain_ckpt["callbacks"]["ModelCheckpoint{'monitor': 'val-totloss', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None, 'save_on_train_epoch_end': True}"].keys()

dict_keys(['monitor', 'best_model_score', 'best_model_path', 'current_score', 'dirpath', 'best_k_models', 'kth_best_model_path', 'kth_value', 'last_model_path'])

In [34]:
zenodo_TS1x_pretrain_ckpt["callbacks"]["ModelCheckpoint{'monitor': 'val-totloss', 'mode': 'min', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None, 'save_on_train_epoch_end': True}"]

{'monitor': 'val-totloss',
 'best_model_score': None,
 'best_model_path': '',
 'current_score': None,
 'dirpath': '/home/ubuntu/efs/demo_OA_ReactDiff/oa_reactdiff/trainer/checkpoint/TSDiff/leftnet-10-ca9dab2855a6',
 'best_k_models': {},
 'kth_best_model_path': '',
 'kth_value': tensor(inf, dtype=torch.float32),
 'last_model_path': ''}

In [46]:
len(zenodo_TS1x_pretrain_ckpt['optimizer_states'])

1

In [40]:
zenodo_TS1x_pretrain_ckpt['optimizer_states'][0].keys()

dict_keys(['state', 'param_groups'])

In [53]:
{k: v.shape for k, v in zenodo_TS1x_pretrain_ckpt['optimizer_states'][-1]["state"][0].items()}

{'step': torch.Size([]),
 'exp_avg': torch.Size([196, 8]),
 'exp_avg_sq': torch.Size([196, 8]),
 'max_exp_avg_sq': torch.Size([196, 8])}

In [45]:
zenodo_TS1x_pretrain_ckpt['optimizer_states'][-1]["param_groups"]

[{'lr': 0.0005,
  'betas': [0.9, 0.999],
  'eps': 1e-08,
  'weight_decay': 0,
  'amsgrad': True,
  'foreach': None,
  'maximize': False,
  'capturable': False,
  'params': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   1

In [54]:
zenodo_TS1x_pretrain_ckpt['lr_schedulers']

[]

In [55]:
zenodo_TS1x_pretrain_ckpt['hparams_name']

'kwargs'

In [59]:
z_conf = zenodo_TS1x_pretrain_ckpt['hyper_parameters']["model_config"]

In [57]:
# OK, compare this to what we have done then:

In [60]:
fromscratch_TS1x_pretrain_ckpt = torch.load("./trained_models/pretrained-ts1x-diff-from-scratch.ckpt", map_location="cpu")
fs_conf = fromscratch_TS1x_pretrain_ckpt["hyper_parameters"]["model_config"]

In [62]:
for i, (zkv, fskv) in enumerate(zip(z_conf.items(), fs_conf.items())):
    zk, zv = zkv
    fsk, fsv = fskv
    assert zk == fsk
    if zv != fsv:
        print(f"Difference found in {zk}: {zv} vs {fsv}.")

In [78]:
fs_opt_pgroups = fromscratch_TS1x_pretrain_ckpt['optimizer_states'][0]["param_groups"][0]

In [79]:
z_opt_pgroups = zenodo_TS1x_pretrain_ckpt["optimizer_states"][0]["param_groups"][0]

In [80]:
z_opt_pgroups

{'lr': 0.0005,
 'betas': [0.9, 0.999],
 'eps': 1e-08,
 'weight_decay': 0,
 'amsgrad': True,
 'foreach': None,
 'maximize': False,
 'capturable': False,
 'params': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  1

In [81]:
for i, (zkv, fskv) in enumerate(zip(z_opt_pgroups.items(), fs_opt_pgroups.items())):
    zk, zv = zkv
    fsk, fsv = fskv
    assert zk == fsk
    if zv != fsv:
        print(f"Difference found in {zk}: {zv} vs {fsv}.")

Difference found in lr: 0.0005 vs 0.00025.


In [ ]:
zenodo_TS1x_pretrain_ckpt['optimizer_states'][-1]["param_groups"]

# Resume pre-train sampling workflow:

In [7]:
noise_schedule: str = "polynomial_2"
timesteps: int = 150
precision: float = 1e-5

gamma_module = PredefinedNoiseSchedule(
            noise_schedule=noise_schedule,
            timesteps=timesteps,
            precision=precision,
        )
schedule = DiffSchedule(
    gamma_module=gamma_module,
    norm_values=ddpm_trainer.ddpm.norm_values
)
ddpm_trainer.ddpm.schedule = schedule
ddpm_trainer.ddpm.T = timesteps
ddpm_trainer = ddpm_trainer.to(device)

AttributeError: 'ConfModule' object has no attribute 'ddpm'


### Prepare dataset and data loader and select a reaction involving multiple molecules

In [ ]:
import pickle

npz_path = "./oa_reactdiff/data/transition1x/train_addprop.pkl" # w/o self-loops after Angstrom correction

train_pkl = pickle.load(open(npz_path, "rb"))

In [ ]:
train_pkl["use_ind"][:10]

In [ ]:
train_pkl["reactant"]["rxn"][177]

In [ ]:
train_pkl["reactant"]['num_atoms'][177]

In [ ]:
dataset = ProcessedTS1x(
    npz_path=npz_path,
    center=True,
    pad_fragments=0,
    device=device,
    zero_charge=False,
    remove_h=False,
    single_frag_only=False,
    swapping_react_prod=False,
    use_by_ind=True,
)
loader = DataLoader(
    dataset, 
    batch_size=1,
    shuffle=False,
    num_workers=0,
    collate_fn=dataset.collate_fn
)
itl = iter(loader)
idx = -1 # TL: why?

len(dataset)

In [ ]:
num_indices_to_select = 20
SEED = 747
np.random.default_rng(seed=SEED)
np.random.seed(SEED)
random_indices = np.random.choice(len(dataset), size=num_indices_to_select, replace=False)
random_indices = sorted(random_indices)
random_indices

In [ ]:
train_indices = [train_pkl["use_ind"][x] for x in random_indices]
train_indices

In [ ]:
# TL visualize each state:
atomic_num2sym = {
    1: 'H',    2: 'He',   3: 'Li',   4: 'Be',   5: 'B',    6: 'C',    7: 'N',    8: 'O',    9: 'F',    10: 'Ne',
    11: 'Na',  12: 'Mg',  13: 'Al',  14: 'Si',  15: 'P',   16: 'S',   17: 'Cl',  18: 'Ar',  19: 'K',   20: 'Ca',
    21: 'Sc',  22: 'Ti',  23: 'V',   24: 'Cr',  25: 'Mn',  26: 'Fe',  27: 'Co',  28: 'Ni',  29: 'Cu',  30: 'Zn',
    31: 'Ga',  32: 'Ge',  33: 'As',  34: 'Se',  35: 'Br',  36: 'Kr',  37: 'Rb',  38: 'Sr',  39: 'Y',   40: 'Zr',
    41: 'Nb',  42: 'Mo',  43: 'Tc',  44: 'Ru',  45: 'Rh',  46: 'Pd',  47: 'Ag',  48: 'Cd',  49: 'In',  50: 'Sn',
    51: 'Sb',  52: 'Te',  53: 'I',   54: 'Xe',  55: 'Cs',  56: 'Ba',  57: 'La',  58: 'Ce',  59: 'Pr',  60: 'Nd',
    61: 'Pm',  62: 'Sm',  63: 'Eu',  64: 'Gd', 65: 'Tb',  66: 'Dy',  67: 'Ho',  68: 'Er',  69: 'Tm',  70: 'Yb',
    71: 'Lu',  72: 'Hf',  73: 'Ta',  74: 'W',   75: 'Re',  76: 'Os',  77: 'Ir',  78: 'Pt',  79: 'Au',  80: 'Hg',
    81: 'Tl',  82: 'Pb',  83: 'Bi',  84: 'Po',  85: 'At',  86: 'Rn',  87: 'Fr',  88: 'Ra',  89: 'Ac',  90: 'Th',
    91: 'Pa',  92: 'U',   93: 'Np',  94: 'Pu',  95: 'Am',  96: 'Cm',  97: 'Bk',  98: 'Cf',  99: 'Es', 100: 'Fm',
    101: 'Md', 102: 'No', 103: 'Lr', 104: 'Rf', 105: 'Db', 106: 'Sg', 107: 'Bh', 108: 'Hs', 109: 'Mt', 110: 'Ds',
    111: 'Rg', 112: 'Cn', 113: 'Nh', 114: 'Fl', 115: 'Mc', 116: 'Lv', 117: 'Ts', 118: 'Og'
}

In [ ]:
def xyz_block_from_node_features(xh: torch.tensor, comment: str="", c2a: dict=atomic_num2sym) -> str:
    num_atoms = xh.shape[0]
    xyz_lines = [str(num_atoms), comment]
    for row in xh:
        position = row[:3].cpu().numpy()
        z = c2a[row[-1].long().item()]
        xyz_lines.append(f"{z}\t" + "\t".join([str(x) for x in position]))
    return "\n".join(xyz_lines)

In [ ]:
!ls -haltr results

In [ ]:
output_dir = os.path.abspath("results/sample_20_TS-20250617-TS1x-from_scratch_confidence-epoch1999_ckpt")
#output_dir = None

In [ ]:
selected_representation_triples = {}
rs_rmsdsx = []
ts_rmsdsx = []
ps_rmsdsx = []

for i in range(len(dataset)): 
    representations, res = next(itl)
    if i in random_indices:
        xyz_blocks = []
        print(i)
        train_idx = train_pkl["use_ind"][i]
        print(train_idx)
        rxn_id = train_pkl["reactant"]["rxn"][train_idx]
        n_samples = representations[0]["size"].size(0)
        fragments_nodes = [
            repre["size"] for repre in representations
        ]
        conditions = torch.tensor([[0] for _ in range(n_samples)], device=device)
        # skipping permutation of indices in reactant state
        xh_fixed = [
            torch.cat(
                [repre[feature_type] for feature_type in FEATURE_MAPPING],
                dim=1,
            )
            for repre in representations
        ]
        print(xh_fixed[2].shape[0])
        print(train_pkl["reactant"]['num_atoms'][train_idx])
        #ground_truth_ts = xh_fixed[1]
        out_samples, out_masks = ddpm_trainer.ddpm.inpaint(
            n_samples=n_samples,
            fragments_nodes=fragments_nodes,
            conditions=conditions,
            return_frames=1,
            resamplings=5,
            jump_length=5,
            timesteps=None,
            xh_fixed=xh_fixed,
            frag_fixed=[0, 2],
        )
        
        # Confirm unchanged reactant state: torch.allclose causing segmentation fault?
        #reactant_diff = torch.allclose(out_samples[0][0] - xh_fixed[0]) #= torch.max(torch.abs(out_samples[0][0] - xh_fixed[0]))
        #print(reactant_diff)
        #assert out_samples[0][0] == xh_fixed[0]
        # Confirm unchanged product state:
        #assert out_samples[0][2] == xh_fixed[2]
        # reactant state (rs_..):
        rs_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=0,
        )
        print(rs_rmsds)
        rs_rmsdsx.append(rs_rmsds[0])

        # transition state (ts_..):
        ts_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=1,
        )
        print(ts_rmsds)
        ts_rmsdsx.append(ts_rmsds[0])
        
        # product state (ps_..):
        ps_rmsds = batch_rmsd(
            fragments_nodes, 
            out_samples[0],
            xh_fixed,
            idx=2,
        )
        print(ps_rmsds)
        ps_rmsdsx.append(ps_rmsds[0])
        #assert len(rmsds) == 1
        print("")

        if output_dir == None:
            continue # skip preparing strings for file output

        # Now wrap up XYZs into an output file with informative comments.
        file_name = f"{rxn_id}.xyz"
        # Reactant state, two versions
        rs_ref_xyz = xyz_block_from_node_features(xh_fixed[0], comment=f"True/calculated reference reactant state.")
        xyz_blocks.append(rs_ref_xyz)
        rs_rec_xyz = xyz_block_from_node_features(out_samples[0][0], comment=f"Reconstructed reactant state. RMSD: {str(round(rs_rmsds[0],6))} Å.")
        xyz_blocks.append(rs_rec_xyz)
        
        # Transition state, two versions
        ts_ref_xyz = xyz_block_from_node_features(xh_fixed[1], comment=f"True/calculated reference transition state.")
        xyz_blocks.append(ts_ref_xyz)
        ts_gen_xyz = xyz_block_from_node_features(out_samples[0][1], comment=f"Generated/inpainted transition state. RMSD: {str(round(ts_rmsds[0],6))} Å.")
        xyz_blocks.append(ts_gen_xyz)

        # Product state, two versions
        ps_ref_xyz = xyz_block_from_node_features(xh_fixed[2], comment=f"True/calculated reference product state.")
        xyz_blocks.append(ps_ref_xyz)
        ps_rec_xyz = xyz_block_from_node_features(out_samples[0][2], comment=f"Reconstructed product state. RMSD: {str(round(ps_rmsds[0],6))} Å.")
        xyz_blocks.append(ps_rec_xyz)

        with open(os.path.join(output_dir, file_name), "w") as f_out:
            f_out.write("\n".join(xyz_blocks))
        
        if i == random_indices[-1]: # no need to keep iterating.
            break

In [ ]:
# Report/analysis of RMSDs seen:

rs_mean = round(np.mean(rs_rmsdsx), 6)
rs_std = round(np.std(rs_rmsdsx), 6)
print(f"Reactant state reconstruction RMSD was mean ± std.dev.: \t{rs_mean} \t± {rs_std} Å.")
ts_mean = round(np.mean(ts_rmsdsx), 6)
ts_std = round(np.std(ts_rmsdsx), 6)
print(f"Transition state inpainting RMSD was mean ± std.dev.: \t\t{ts_mean} \t± {ts_std} Å.")
ps_mean = round(np.mean(ps_rmsdsx), 6)
ps_std = round(np.std(ps_rmsdsx), 6)
print(f"Product state reconstruction RMSD was mean ± std.dev.: \t\t{ps_mean} \t± {ps_std} Å.")